# Receipt Data Extraction: Extracting Structured Data from Ad-Campaign Receipts

## Overview
This notebook demonstrates how to use Snowflake's Document AI capabilities to automatically extract structured information from ad-campaign receipt PDFs. We'll process receipts uploaded to our Snowflake stage and transform them into queryable structured data.

## What We'll Accomplish
- Parse PDF receipts from the `@RECEIPTS_PROCESSING_DB.RAW.RECEIPTS` stage using `AI_PARSE_DOCUMENT`
- Extract structured receipt data (vendor, transaction details, campaign info, metrics) using `AI_COMPLETE` with a defined schema
- Transform unstructured receipt content into structured, analyzable data

## Prerequisites
- Access to Snowflake with Cortex AI features enabled
- Receipts uploaded to `@RECEIPTS_PROCESSING_DB.RAW.RECEIPTS` stage
- RECEIPTS_PROCESSING_DB database and RAW schema configured
- Appropriate permissions for the ETL service role


## Step 1: Environment Setup and Session Initialization

Setting up our environment by importing necessary packages and establishing a Snowflake session.

### Key Components:
- **Streamlit**: For building interactive applications
- **Pandas**: For data manipulation and analysis
- **Snowpark Session**: Connection to Snowflake and access to Cortex AI capabilities

The `get_active_session()` retrieves our Snowflake session for executing SQL and using AI features.


In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# Use Snowpark for our analyses
from snowflake.snowpark.context import get_active_session
session = get_active_session()


## Step 2: Set Database and Schema Context

Setting the working context for our session:

- **Database**: `RECEIPTS_PROCESSING_DB` - Our receipt processing database
- **Schema**: `RAW` - The schema containing our receipts stage

This ensures all operations execute within the correct context without needing to fully qualify object names.


In [ ]:
session.sql("USE ROLE SYSADMIN").collect()

# Set warehouse for AI processing
session.sql("USE WAREHOUSE RECEIPTS_PARSE_COMPLETE_WH").collect()

session.sql("ALTER WAREHOUSE RECEIPTS_PARSE_COMPLETE_WH SET WAREHOUSE_SIZE='XSMALL'").collect()

session.use_database('RECEIPTS_PROCESSING_DB')
session.use_schema('RAW')


## Step 3: Import AI and Processing Libraries

Importing specialized libraries for document AI processing:

### Key Imports:
- **Pydantic**: For data validation and schema definition
- **Snowflake Cortex**: Access to AI functions (`complete`, `CompleteOptions`)
- **Snowpark Functions**: SQL functions including `ai_complete` and `prompt`
- **JSON**: For handling structured data
- **Typing**: For type hints (List, Optional)

These libraries enable us to process receipt documents and extract structured information using large language models.


In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional
#import json
#from snowflake.cortex import complete, CompleteOptions
#from snowflake.snowpark.functions import col, prompt, ai_complete


## Step 4: Explore Available Receipts

Before processing, let's see what receipt files are available in our stage.

### What This Shows:
- File names and paths of receipts ready for processing
- File sizes and metadata
- Upload timestamps

The `DIRECTORY()` function provides a view of all files in the `@RECEIPTS_PROCESSING_DB.RAW.RECEIPTS` stage, essential for understanding our data source.


In [ ]:
--REMOVE @RECEIPTS_PROCESSING_DB.RAW.RECEIPTS; -- REMOVES ALL FILES FROM THE STAGE
ALTER STAGE RECEIPTS_PROCESSING_DB.RAW.RECEIPTS REFRESH;
SELECT * FROM DIRECTORY(@RECEIPTS_PROCESSING_DB.RAW.RECEIPTS);


In [ ]:
-- clean up the table if we want to re-parse documents:
--truncate table parsed_receipts;

## Step 5: Parse Receipt PDFs with AI_PARSE_DOCUMENT (Incremental)

Using Snowflake's AI to extract text content from receipt PDFs - only parsing new documents.

### What's Happening:
1. **Create Table If Not Exists**: Creates `parsed_receipts` table on first run (not transient)
2. **Incremental Processing**: Only parses documents NOT already in `parsed_receipts` table
3. **AI_PARSE_DOCUMENT**: This function:
   - Reads PDF files from `@RECEIPTS_PROCESSING_DB.RAW.RECEIPTS` stage
   - Uses `'layout'` mode to preserve receipt structure
   - Extracts text content including vendor info, line items, amounts, campaign details
4. **INSERT Results**: Adds only new parsed content to existing table

### Benefits of Incremental Processing:
- ✅ Avoids re-parsing already processed documents (saves time and costs)
- ✅ Preserves existing parsed data
- ✅ Only processes new receipts uploaded to stage
- ✅ Can run repeatedly without duplicating work

### Why Layout Mode?
Layout mode preserves the receipt's visual structure (headers, tables, campaign details section), which helps the AI understand:
- Vendor branding and headers
- Line item tables with services and amounts
- Campaign details section (display/video formats, metrics)
- Total amounts and tax calculations


In [ ]:
# Create parsed_receipts table if it doesn't exist
session.sql("""
CREATE TABLE IF NOT EXISTS parsed_receipts (
    relative_path STRING,
    content STRING
)
""").collect()

# Only parse documents that haven't been parsed yet
docs_df = session.sql("""
INSERT INTO parsed_receipts
SELECT
    relative_path,
    AI_PARSE_DOCUMENT(
        to_file('@RECEIPTS_PROCESSING_DB.RAW.RECEIPTS', relative_path), 
        {'mode': 'layout'}
    ):content AS content
FROM DIRECTORY(@RECEIPTS_PROCESSING_DB.RAW.RECEIPTS)
WHERE relative_path NOT IN (SELECT relative_path FROM parsed_receipts)
""").collect()

# Get the actual number of rows inserted from the result metadata
rows_inserted = docs_df[0]['number of rows inserted'] if docs_df else 0
print(f"✓ Parsed {rows_inserted} new receipt(s)")

## Step 6: Preview Parsed Receipt Content

Examining the parsed receipt data:

- **relative_path**: Original receipt filename (e.g., `receipt_TechAds_Pro_20251020.pdf`)
- **content**: Extracted text from the receipt PDF

This verification ensures successful parsing and shows the text quality for extraction.


In [ ]:
#session.table('parsed_receipts').to_pandas()
session.table('parsed_receipts').to_pandas().head()


## Step 7: Define Receipt Extraction Prompt

Creating the instruction prompt to guide AI extraction from receipt documents.

### Prompt Structure:
- **Task Definition**: Clear instructions to analyze receipts and extract specific fields
- **Document Content Placeholder**: `{0}` will be replaced with actual receipt text
- **Output Format**: Explicit JSON structure requirements

### Information to Extract:
1. **Vendor Details**: Name, branding, contact information
2. **Transaction Info**: Receipt ID, date, time, payment method
3. **Customer Info**: Client name and company
4. **Campaign Details**: Name, display/video formats, ad types
5. **Financials**: Line items, subtotal, tax, total amount
6. **Metrics**: CPM, CTR, bounce rate, target impressions/clicks
7. **Targeting**: Geographic, demographic, device targeting

This prompt instructs the AI model on exactly what data to extract and how to format the response.


In [ ]:
prompt_text = """
    CONCAT($$Analyze the ad-campaign receipt document provided and extract the following structured information:
    
    1. Vendor/Provider details (name, contact info)
    2. Transaction information (ID, date, time, payment method)
    3. Customer/Client information (name, company)
    4. Campaign details (name, content types, ad formats, period start date, period end date)
    5. Financial details (line items, subtotal, tax, total)
    6. Campaign metrics (CPM, CTR, bounce rate %, targets)
    7. Targeting information (geography, demographics, devices)
    
    <document-content>$$, {0}, $$
    </document-content>

    <output-format>
    Provide JSON output in the exact format specified in the response schema.
    For any fields not found in the document, use an empty string.
    For numeric fields, use numbers (not strings).
    For percentages, use whole numbers.
    For arrays, provide empty arrays [] if no data is found.
    </output-format>$$)
"""


In [ ]:
# This prompt is for receipts of the 'v2' type (multiple tables of the same type with line items within them)
prompt_text_v2 = """
    CONCAT($$Analyze the ad-campaign receipt document provided and extract the following structured information:
    
    1. Vendor/Provider details (name)
    2. Transaction information (ID, date, payment method)
    3. Customer/Client information (name, company)
    4. Campaign name
    5. PRICING section containing multiple pricing tables:
       - Each table has a NAME (e.g., "Geographic Pricing", "Demographic Pricing", etc.)
       - Each table contains MARKETS with at least the following columns:
         * Market/Region name
         * Minimum (USD) - minimum spend value
         * Reach - reach number (impressions/audience size)
    6. Financial totals (subtotal, tax, total)
    
    <document-content>$$, {0}, $$
    </document-content>

    <output-format>
    Provide JSON output in the exact format specified in the response schema.
    For the pricing_tables array, extract ALL tables found in the PRICING section.
    Each table should include its name and all market rows.
    For any fields not found in the document, use an empty string or empty array.
    For numeric fields, preserve the values as strings to maintain formatting.
    </output-format>$$)
"""


## Step 8: Define Response Schema for Receipt Extraction

Defining the exact structure for extracted receipt data with all receipt-specific fields including vendor, transaction, campaign details, metrics (CPM, CTR, Bounce Rate), budget, targeting, and line items.


In [ ]:
resp_schema = """
{
    'type': 'json',
    'schema': {
        'type': 'object',
        'properties': {
            'vendor': {
                'type': 'object',
                'properties': {
                    'vendor_name': {'type': 'string'}
                },
                'required': ['vendor_name']
            },
            'transaction': {
                'type': 'object',
                'properties': {
                    'receipt_id': {'type': 'string'},
                    'date': {'type': 'string'},
                    'payment_method': {'type': 'string'}
                },
                'required': ['receipt_id', 'date', 'payment_method']
            },
            'customer': {
                'type': 'object',
                'properties': {
                    'customer_name': {'type': 'string'},
                    'company_name': {'type': 'string'}
                },
                'required': ['company_name']
            },
            'campaign': {
                'type': 'object',
                'properties': {
                    'name': {'type': 'string'},
                    'content_types': {'type': 'string'},
                    'ad_formats': {'type': 'array'},
                    'period_startdate': {'type': 'string'},
                    'period_enddate': {'type': 'string'},
                    'budget': {'type':'number'}
                },
                'required': ['name', 'content_types', 'ad_formats', 'period_startdate', 'period_enddate']
            },
            'financials': {
                'type': 'object',
                'properties': {
                    'line_items': {'type': 'object'},
                    'subtotal': {'type': 'number'},
                    'tax': {'type': 'number'},
                    'total': {'type': 'number'}
                },
                'required': ['total', 'line_items', 'subtotal', 'tax']
            },
            'metrics': {
                'type': 'object',
                'properties': {
                    'cpm': {'type': 'string', 'description':'Cost Per Milli, abbreviated as CPM'},
                    'ctr': {'type': 'string', 'description':'Click-through rate, abbreviated as CTR'},
                    'bounce_rate': {'type': 'string', 'description':'Bounce Rate, sometimes just referred to as Bounce, a % value'},
                    'targets': {'type': 'object'},
                    'pricing_model': {'type': 'string'}
                },
                'required': ['cpm', 'ctr', 'bounce_rate', 'targets', 'pricing_model']
            },
            'budget':{
                'type': 'object',
                'properties': {
                    'daily_budget': {'type': 'string'},
                    'total_budget': {'type': 'string'}
                },
                'required': ['daily_budget', 'total_budget']
            },
            'targeting': {
                'type': 'object',
                'properties': {
                    'geography': {'type': 'array'},
                    'demographics': {'type': 'string'},
                    'age_range': {'type': 'string'},
                    'devices': {'type': 'string'}
                },
                'required': ['geography', 'demographics', 'age_range', 'devices']
            }
        },
        'required': ['vendor', 'transaction', 'customer', 'campaign', 'financials', 'metrics', 'budget', 'targeting']
    }
}
"""


In [ ]:
# This response schema is for receipts of the 'v2' type (multiple tables of the same type with line items within them)
resp_schema_v2 = """
{
    'type': 'json',
    'schema': {
        'type': 'object',
        'properties': {
            'vendor': {
                'type': 'object',
                'properties': {
                    'vendor_name': {'type': 'string'}
                },
                'required': ['vendor_name']
            },
            'transaction': {
                'type': 'object',
                'properties': {
                    'transaction_id': {'type': 'string'},
                    'date': {'type': 'string'},
                    'payment_method': {'type': 'string'}
                },
                'required': ['transaction_id', 'date', 'payment_method']
            },
            'customer': {
                'type': 'object',
                'properties': {
                    'customer_name': {'type': 'string'},
                    'company_name': {'type': 'string'}
                },
                'required': ['company_name']
            },
            'campaign': {
                'type': 'object',
                'properties': {
                    'name': {'type': 'string'}
                },
                'required': ['name']
            },
            'pricing_tables': {
                'type': 'array',
                'description': 'Array of pricing tables found in the PRICING section',
                'items': {
                    'type': 'object',
                    'properties': {
                        'table_name': {
                            'type': 'string',
                            'description': 'Name of the pricing table (e.g., Geographic Pricing, Demographic Pricing)'
                        },
                        'markets': {
                            'type': 'array',
                            'description': 'List of markets in this pricing table',
                            'items': {
                                'type': 'object',
                                'properties': {
                                    'market': {
                                        'type': 'string',
                                        'description': 'Market or region name'
                                    },
                                    'minimum_usd': {
                                        'type': 'string',
                                        'description': 'Minimum spend value in USD'
                                    },
                                    'reach': {
                                        'type': 'string',
                                        'description': 'Reach number (impressions/audience)'
                                    }
                                },
                                'required': ['market', 'minimum_usd', 'reach']
                            }
                        }
                    },
                    'required': ['table_name', 'markets']
                }
            },
            'financials': {
                'type': 'object',
                'properties': {
                    'subtotal': {'type': 'number'},
                    'tax': {'type': 'number'},
                    'total': {'type': 'number'}
                },
                'required': ['subtotal', 'tax', 'total']
            }
        },
        'required': ['vendor', 'transaction', 'customer', 'campaign', 'pricing_tables', 'financials']
    }
}
"""


In [ ]:
# Pydantic-based schema generation for V2 receipts (pricing tables)
from pydantic import BaseModel, Field
from typing import List
import json

# Define nested models matching resp_schema_v2 structure
class Vendor(BaseModel):
    """Vendor information."""
    vendor_name: str = Field(description="Name of the vendor")

class Transaction(BaseModel):
    """Transaction details."""
    transaction_id: str = Field(description="Transaction or invoice ID")
    date: str = Field(description="Date of the transaction")
    payment_method: str = Field(description="Payment method used")

class Customer(BaseModel):
    """Customer information."""
    customer_name: str = Field(description="Name of the customer contact")
    company_name: str = Field(description="Name of the customer company")

class Campaign(BaseModel):
    """Campaign details."""
    name: str = Field(description="Name of the advertising campaign")

class Market(BaseModel):
    """Individual market within a pricing table."""
    market: str = Field(description="Market or region name")
    minimum_usd: str = Field(description="Minimum spend value in USD")
    reach: str = Field(description="Reach number (impressions/audience)")

class PricingTable(BaseModel):
    """Single pricing table with name and markets."""
    table_name: str = Field(description="Name of the pricing table (e.g., Geographic Pricing)")
    markets: List[Market] = Field(description="List of markets in this pricing table")

class Financials(BaseModel):
    """Financial totals."""
    subtotal: str = Field(description="Subtotal amount")
    tax: str = Field(description="Tax amount")
    total: str = Field(description="Total amount charged")

class ReceiptV2(BaseModel):
    """Complete V2 receipt with pricing tables."""
    vendor: Vendor
    transaction: Transaction
    customer: Customer
    campaign: Campaign
    pricing_tables: List[PricingTable] = Field(description="Array of pricing tables found in the PRICING section")
    financials: Financials

# Generate AI_COMPLETE compatible schema WITH $defs and $ref
# Snowflake AI_COMPLETE supports $defs and $ref (see Snowflake docs)
resp_schema_v2b = {
    'type': 'json',
    'schema': ReceiptV2.model_json_schema()
}

print("Pydantic-based V2 schema (resp_schema_v2b) generated with $defs and $ref")
print(f"Schema has {len(resp_schema_v2b['schema']['properties'])} top-level properties")
print(f"Schema has {len(resp_schema_v2b['schema'].get('$defs', {}))} definitions in $defs")
# Uncomment to view full schema:
# print(json.dumps(resp_schema_v2b, indent=2))

### Pydantic Schema Generation

**resp_schema_v2b** uses Pydantic to programmatically generate the JSON schema.

**Advantages**:
- ✅ **Type Safety**: Python classes with proper type hints
- ✅ **Maintainable**: Modify Python classes instead of raw JSON
- ✅ **Validated**: Pydantic validates structure automatically
- ✅ **Documented**: Docstrings and Field descriptions
- ✅ **Reusable**: Can use models for data validation elsewhere

**About \$defs and \$ref**:
Pydantic generates schemas with JSON Schema references (\$defs and \$ref). This is **fully supported** by Snowflake AI_COMPLETE and creates cleaner, more modular schemas.

**Model Hierarchy**:
```
ReceiptV2
├── vendor: Vendor (vendor_name)
├── transaction: Transaction (transaction_id, date, payment_method)
├── customer: Customer (customer_name, company_name)
├── campaign: Campaign (name)
├── pricing_tables: List[PricingTable]
│   └── PricingTable
│       ├── table_name
│       └── markets: List[Market]
│           └── Market (market, minimum_usd, reach)
└── financials: Financials (subtotal, tax, total)
```

**Comparison**:
- `resp_schema_v2`: Hand-written JSON (simple, inline, no \$refs)
- `resp_schema_v2b`: Pydantic-generated (modular, with \$defs/\$refs)

Both produce functionally equivalent schemas for AI_COMPLETE.


### V2 Schema for Pricing Tables

The `prompt_text_v2` and `resp_schema_v2` are designed for receipts from **receipts.synthesis_v2** which use pricing tables instead of line items.

**Key Differences**:
- ✅ **pricing_tables**: Array of table objects (instead of line_items)
- ✅ **table_name**: Each table has a name (e.g., "Geographic Pricing")
- ✅ **markets**: Array of market objects with market, minimum_usd, reach
- ✅ **Nested structure**: Supports multiple tables, each with multiple markets

**Usage**: 
To process V2 receipts, replace `{prompt_text}` with `{prompt_text_v2}` and `{resp_schema}` with `{resp_schema_v2}` in the query definition.

**Example V2 extraction**:
```json
{
  "pricing_tables": [
    {
      "table_name": "Geographic Pricing",
      "markets": [
        {"market": "North America", "minimum_usd": "$25,000", "reach": "5,000,000"},
        {"market": "Europe", "minimum_usd": "$18,000", "reach": "3,200,000"}
      ]
    },
    {
      "table_name": "Demographic Pricing",
      "markets": [...]
    }
  ]
}
```


## Step 9: Extract Structured Data with AI_COMPLETE (Incremental)

Using Snowflake's AI to extract structured receipt data - only processing newly parsed receipts.

### Incremental Extraction:
- **CREATE TABLE IF NOT EXISTS**: Preserves existing extracted data
- **INSERT INTO**: Adds only new extractions
- **WHERE NOT IN**: Only processes receipts not already in extracted_receipt_data
- **Saves Costs**: Avoids re-running expensive AI_COMPLETE on same receipts

The AI reads each new receipt and extracts vendor details, transaction info, campaign details (display/video formats), financial totals, performance metrics (CPM, CTR, Bounce Rate), targeting parameters, and line items into structured JSON.


In [ ]:
-- clean up the table if we want to re-extract documents:
--truncate table extracted_receipt_data;

In [ ]:
# Create extracted_receipt_data table if it doesn't exist
session.sql("""
CREATE TABLE IF NOT EXISTS extracted_receipt_data (
    relative_path STRING,
    content STRING,
    extracted_data VARIANT
)
""").collect()

# Only extract data from newly parsed receipts
# Try different models, like 'claude-sonnet-4-5' or 'claude-haiku-4-5' (in PuPr for cross-region availability)
query = f"""
INSERT INTO extracted_receipt_data
SELECT
    relative_path,
    content,
    ai_complete(
        model=>'claude-haiku-4-5',
        prompt=>{prompt_text_v2.format('content')},
        response_format=>{resp_schema_v2b}
    ) as extracted_data
FROM parsed_receipts
WHERE relative_path NOT IN (SELECT relative_path FROM extracted_receipt_data)
"""

session.sql(query).collect()

## Step 10: Preview Extracted Receipt Data

Each row contains a complete structured representation of a receipt with all extracted fields in JSON format, ready for flattening and analysis.


In [ ]:
# View the extracted data
result_df = session.table('extracted_receipt_data').to_pandas()
result_df.head()


In [ ]:
# Example analytics: Spending by vendor
session.sql("""
SELECT 
    vendor_name,
    COUNT(*) AS receipt_count,
    SUM(total_amount) AS total_spending,
    AVG(total_amount) AS avg_receipt_amount,
    AVG(cpm) AS avg_cpm,
    AVG(ctr_percent) AS avg_ctr
FROM receipt_analytics_vw
GROUP BY vendor_name
ORDER BY total_spending DESC
""").to_pandas()


In [ ]:
# Campaign type analysis
session.sql("""
SELECT 
    content_types,
    COUNT(*) AS campaign_count,
    AVG(total_amount) AS avg_spending,
    AVG(cpm) AS avg_cpm,
    AVG(ctr_percent) AS avg_ctr,
    AVG(bounce_rate_percent) AS avg_bounce_rate
FROM receipt_analytics_vw
WHERE content_types IS NOT NULL
GROUP BY content_types
ORDER BY campaign_count DESC
""").to_pandas()


In [ ]:
# Performance metrics by pricing model
session.sql("""
SELECT 
    pricing_model,
    COUNT(*) AS receipt_count,
    AVG(cpm) AS avg_cpm,
    AVG(ctr_percent) AS avg_ctr,
    AVG(bounce_rate_percent) AS avg_bounce_rate,
    SUM(total_amount) AS total_spending
FROM receipt_analytics_vw
WHERE pricing_model IS NOT NULL
GROUP BY pricing_model
ORDER BY receipt_count DESC
""").to_pandas()


## Summary

### What We've Accomplished:

1. ✅ **Parsed Receipts**: Extracted text from PDF receipts using AI_PARSE_DOCUMENT
2. ✅ **Structured Extraction**: Converted unstructured receipts to structured JSON using AI_COMPLETE
3. ✅ **Stored Data**: Populated `extracted_receipt_data` table with structured receipt information

### Tables Created:
1. `parsed_receipts` - Raw parsed text from PDFs
2. `extracted_receipt_data` - Structured JSON extraction

### For Analytics:
Run `receipts-analysis/analysis.sql` which will:
- Create `receipt_analytics_vw` (flattened view)
- Provide ready-to-use analytical queries

---

**Your receipt data is now extracted and ready for analytics!** 📊
